# Your Title Information Goes Here
Fill me in

## Setup

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Project-Template-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5660-Project-Template-F23`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Project-Template-F23/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5660-Project-Template-F23/Project.toml`
    Updating `~/Desktop/julia_work/CHEME-5660-Project-Template-F23/Manifest.toml`
  [6e4b80f9] ↑ BenchmarkTools v1.3.2 ⇒ v1.4.0
  [1a297f60] ↑ FillArrays v1.8.0 ⇒ v1.9.2
  [4076af6c] ↑ JuMP v1.16.0 ⇒ v1.17.0
  [1d5cc7b8] ↑ Intel

## Prerequisites: Load historical dataset, compute expected returns and get 2023 `SPY` data
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) since `01-03-2018` until `12-01-2023`, along with data for a few exchange traded funds and volatility products during that time. 

In [2]:
original_dataset = load(joinpath(_PATH_TO_DATA, 
        "SP500-Daily-OHLC-1-3-2018-to-12-01-2023.jld2")) |> x-> x["dataset"];

### Clean the data
Not all of the tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquistion or de-listing events. Let's collect only those tickers with the maximum number of trading days.

* First, let's compute the number of records for a company that we know has a maximim value, e.g., `AAPL` and save that value in the `maximum_number_trading_days` variable:

In [3]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow;

Now, lets iterate through our data and collect only those tickers that have `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [4]:
dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset;

Let's get a list of firms that we have in cleaned up `dataset`, and save it in the `all_tickers` array:

In [5]:
all_tickers = keys(dataset) |> collect |> sort;
K = length(all_tickers);

### Get the 2023 `SPY` data

In [6]:
startdate = Date(2023,01,03);
SPY_dataset = dataset["SPY"];
SPY_df = filter(:timestamp => x-> x >= startdate, SPY_dataset);

### Compute the expected return for all firms in the dataset
The expected return $\mathbb{E}(r_{i})$ and covariance matrix $\Sigma$ will be used in our calculations, so we'll provide values for both of these items for the entire data set (all `N = 459` tickers), and then you can pick out which tickers you are interested in. 

* First, we compute the expected (annualized) log return by passing the `dataset` and the entire list of firms we have in the dataset (held in the $N\times{1}$ `all_array` array) to the `log_return_matrix(...)` method. The result is stored in the `all_firms_return_matrix` variable, a $T-1\times{N}$ array of log return values. Each row of `all_firms_return_matrix` corresponds to a time-value, while each column corresponds to a firm:

In [7]:
all_firms_return_matrix = log_return_matrix(dataset, all_tickers, 
    Δt = (1.0/252.0), risk_free_rate = 0.0);

## Your project starts here ....

In [8]:
## Fill me in

In [9]:
dataset["QQQ"]

Row,volume,volume_weighted_average_price,open,close,high,low,timestamp,number_of_transactions
,Float64,Float64,Float64,Float64,Float64,Float64,DateTime,Int64
1,2.93836e7,159.666,158.61,160.03,160.17,158.61,2018-01-03T05:00:00,89156
2,2.47761e7,160.368,160.59,160.31,160.79,160.083,2018-01-04T05:00:00,84937
3,2.69923e7,161.426,161.04,161.92,162.03,160.77,2018-01-05T05:00:00,81821
4,2.31591e7,162.279,161.9,162.55,162.63,161.86,2018-01-08T05:00:00,61794
5,2.47824e7,162.607,162.85,162.56,163.04,162.07,2018-01-09T05:00:00,78992
6,3.4004e7,161.881,161.8,162.18,162.21,161.12,2018-01-10T05:00:00,98177
7,2.04735e7,162.763,162.48,163.29,163.3,162.21,2018-01-11T05:00:00,62727
8,3.77627e7,163.871,163.03,164.49,164.66,162.94,2018-01-12T05:00:00,81096
9,4.12318e7,164.867,165.55,164.02,166.415,163.53,2018-01-16T05:00:00,150982


## Disclaimer and Risks
__This content is offered solely for training and  informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy,  is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on your evaluation of your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.